<a href="https://colab.research.google.com/github/cmunozperez/NLP-Python-2025/blob/main/Notebook_1_web_scraping_con_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web scraping
En este notebook veremos algunos aspectos básicos de cómo extraer datos desde una página web.

## Paquetes

Usaremos tres paquetes en particular:

- **BeautifulSoup:** es un paquete que permite explorar código html. Por ejemplo, le podemos pedir que nos diga el contenido correspondiente a una cierta etiqueta.
- **requests:** es un paquete de Python que permite conectarse con sitios web.
- **re:** expresiones regulares (regex)

In [ ]:
from bs4 import BeautifulSoup
import requests
import re

## Primer request

El siguiente código nos permite obtener la información de una página web de *Lingbuzz*. El valor *timeout* indica el tiempo máximo (en segundos) que se debe esperar para una respuesta.

In [ ]:
# Los manuscritos de Lingbuzz se componen de cuatro números precedidos por dos ceros, e.g., 006509.

manuscript_id = '003567'
url = 'https://ling.auf.net/lingbuzz/' + manuscript_id
result = requests.get(url,
                      timeout=10).text

Como pueden ver, lo que obtuvimos es código html análogo al que vimos en Chrome.

In [ ]:
result

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en"><head><title>Bits, bytes, and Unicode: Digital text for linguists - lingbuzz/003567</title><meta http-equiv="Content-Type" content="text/html;charset=utf-8"/><meta http-equiv="Content-Script-Type" content="text/javascript"/><link rel="canonical" href="/lingbuzz/003567"/><meta name="description" content="A linguistically-oriented review of digital text and the representation of text with the Unicode charater set and encoding system. Presents basic terminology and concepts of writing systems, and of di - lingbuzz, the linguistics archive"/><link rel="stylesheet" type="text/css" href="/buzzdocs/styles/article-editor.css"/><link rel="stylesheet" type="text/css" href="/lingbuzz?_s=0HOgdYn10AIh4bMP"/></head><body alink="#111111" vlink="#333344" link="#3333AA" onload="onLoad()">&nbsp;<p></p><center><font si

## Parsear html con BeautifulSoup

Podemos convertir este resultado en un objeto de *BeautifulSoup*, a cuyas subpartes podamos acceder fácilmente.

In [ ]:
doc = BeautifulSoup(result, 'html.parser')

Por ejemplo, ahora podemos acceder fácilmente a todos los *tags* que habíamos visto en el código html.

In [ ]:
for element in doc.find_all():
  print(element.name)

html
head
title
meta
meta
link
meta
link
link
body
p
center
font
b
a
br
a
br
p
table
tr
td
td
a
tr
td
td
br
font
tr
td
td
tr
td
td
tr
td
td
table
tr
p
p
a
a
script


Como vimos antes, buena parte de la información que nos interesa está dentro del tag *center*. Podemos acceder fácilmente a este tag con el atributo adecuado.

In [ ]:
doc.center

<center><font size="+1"><b><a href="/lingbuzz/003567/current.pdf?_s=-8KdXJVswGhnqFdz">Bits, bytes, and Unicode: Digital text for linguists</a></b></font><br/><a href="/lingbuzz/003567?_s=r24qa9hWRt5wRecr&amp;_k=51YkiGSV_EPhX9OA&amp;1">James Crippen</a><br/>January 2016</center>

Tenemos varios métodos que podemos aplicar sobre *doc* para obtener la información que queremos.

```
doc.center.find_all('a') % lista de todos los tags <a> en <center>
doc.center.find('a') % el primer <a> en <center>
doc.center.a % abreviación del método anterior
```

Si agregamos el atributo *.string* sobre uno de estos tags en particular, obtendremos el texto que contienen. Con esto podemos obtener fácilmente el título del artículo y el nombre del autor.

In [ ]:
doc.center.find_all('a')

[<a href="/lingbuzz/003567/current.pdf?_s=-8KdXJVswGhnqFdz">Bits, bytes, and Unicode: Digital text for linguists</a>,
 <a href="/lingbuzz/003567?_s=r24qa9hWRt5wRecr&amp;_k=51YkiGSV_EPhX9OA&amp;1">James Crippen</a>]

In [ ]:
doc.center.a.string

'Bits, bytes, and Unicode: Digital text for linguists'

In [ ]:
doc.center.find_all('a')[1].string

'James Crippen'

Bien, eso es todo lo que necesitamos del tag *center*. Pasemos a *table*, que es el otro tag que contiene información que queremos (palabras clave y número de descargas).

In [ ]:
doc.table

<table cellspacing="15" valign="top"><tr><td>Format: </td><td>[ <a href="/lingbuzz/003567/current.pdf?_s=UCMSIfqJtjoDllVV">pdf</a> ]</td></tr><tr><td>Reference: </td><td>lingbuzz/003567<br/><font size="-1"> (please use that when you cite this article)</font></td></tr><tr><td>Published in: </td><td>University of British Columbia</td></tr><tr><td>keywords: </td><td>unicode, text, orthography, writing systems, human computer interaction, phonology</td></tr><tr><td>Downloaded:</td><td>509 times</td></tr></table>

Veamos qué tags tenemos en ese choclo.

In [ ]:
for num in range(10):
  print(num)

0
1
2
3
4
5
6
7
8
9


In [ ]:
[element.name for element in doc.table]

['tr', 'tr', 'tr', 'tr', 'tr']

Bueno, tenemos seis tags *tr* que representan filas en una tabla (*table rows*). Cada *tr* tiene adentro tags *td* (*table data*) que son celdas de izquierda a derecha. Si nos fijamos en *Lingbuzz*, nos interesan las segundas celdas de la cuarta y sexta (última) filas.

In [ ]:
(doc.table.find_all('tr')[3]).find_all('td')[1].string

'unicode, text, orthography, writing systems, human computer interaction, phonology'

In [ ]:
(doc.table.find_all('tr')[-1]).find_all('td')[1].string

'509 times'

Tal vez sería útil tener un *int* en vez de un *str* con el número de descargas. Lo que hacemos es usar regex para buscar los dígitos en el resultado anterior.

In [ ]:
int(re.search(r'\d+', (doc.table.find_all('tr')[-1]).find_all('td')[1].string).group())

509

## Extraer el abstract

Volvamos a mirar el html. Como se observa, el abstract no se encuentra dentro de ningún tag en particular; está "flotando" dentro de *body* en "sexto lugar".

In [ ]:
list(doc.body)[5]

Sin embargo, encontré casos en los que el abstract está repartido en varias partes sucesivas dentro del html. Por eso, vamos a aplicar una solución de "fuerza bruta" para extraerlo: vamos a tomar todo lo que está entre la sexta posición dentro de *body* y la tabla.

In [ ]:
abstract = ''

for i in list(doc.body)[5:]:
    if i.string == None:
        break
    abstract += i.string

In [ ]:
abstract

'A linguistically-oriented review of digital text and the representation of text with the Unicode charater set and encoding system. Presents basic terminology and concepts of writing systems, and of digital representation of information with binary (and hexadecimal) numbers. Details characters, character encodings, processes of encoding conversion, and file formats. The Unicode character set is discussed in extensive detail, distinguishing code points, character names, meta-structure such as planes and blocks, meta-data such as character types and properties, and basic principles of representation normalization and sorting. Ends with a review of Unicode encoding formats (e.g. UTF-8, UTF-16) and some practical issues for using Unicode in linguistics.'

# A function to rule them all

Pongamos todo lo que hicimos hasta ahora en una única función que (i) toma un *id* de *Lingbuzz* y devuelve una lista con

- el nombre del artículo,
- el nombre sus autores,
- las palabras clave,
- el número de descargas, y
- el abstract.

In [ ]:
def get_page(man_id):
  """
  Given a lingbuzz id number, the function retrieves the title of the manuscript, its author, the date in which it was uploaded, how many times it was downloaded, and its abstract.

  Args:
  manuscript_id (str): a lingbuzz number id

  Returns:
  list: a list with the relevant information of the manuscript
  """

  # Primero se hace el request
  url = 'https://ling.auf.net/lingbuzz/' + man_id
  result = requests.get(url,
                      timeout=10).text

  # Ahora construimos el objeto de BeautifulSuop
  doc = BeautifulSoup(result, 'html.parser')

  # Luego, el título y el autor o autores
  title = doc.center.a.string
  author = doc.center.find_all('a')[1].string

  # Pasemos a la información de la tabla
  keywords = (doc.table.find_all('tr')[3]).find_all('td')[1].string
  downloads = int(re.search(r'\d+', (doc.table.find_all('tr')[-1]).find_all('td')[1].string).group())

  # Por último, el abstract
  abstract = ''

  for i in list(doc.body)[5:]:
    if i.string == None:
        break
    abstract += i.string

  return [man_id, title, author, keywords, downloads, abstract]

In [ ]:
print(get_page('002402'))

['002402', 'Syntacticizing blends: the case of English wh-raising', 'Lieven Danckaert', None, 1568, 'The focus of this paper is the data such as (1), in which a long relativised subject triggers agreement in both the embedded and the matrix clause. \r\n\r\n(1)\tThe company has seen two developments in the standard of hygiene  which are felt  are attributable to the fact that the programme is now fully up and running. \r\n\r\nAt first sight, the example seems to be derived by illicit subject raising from within a finite clause which - at least in English - is standardly unacceptable, regardless of the presence of the complementizer that. However, some  native speaking informants accept data as (1) and such data are also attested. \r\nBecause the pattern in (1) is tied to wh-movement and is unavailable with a regular DP subject we will call the pattern in (1) wh-raising. \r\n\r\nUsing a cartographic framework, we will develop an analysis for the derivation of the data in (1) framed again

# Pandas

Pandas es un paquete de Python que sirve para trabajar con datos en forma de tablas; es un excel para gente grande. Es muy útil para guardar datos como los que acabamos de recolectar.

In [ ]:
import pandas as pd

Llevar a Pandas datos que están almacenados en forma de lista es muy sencillo. A modo de ejemplo, creemos una lista con los datos de dos manuscritos de *Lingbuzz* (que son, en sí mismas, listas).

In [ ]:
# Primero, obtenemos los datos de dos manuscritos
manuscrito1 = get_page('009004')
manuscrito2 = get_page('008762')

# Segundo, creamos una lista con ambos conjuntos de datos
datos = [manuscrito1, manuscrito2]

Una tabla de Pandas se denomina **DataFrame**. Podemos crear un DataFrame con una lista de listas. Como verán, también es posible nombrar cada una de las columnas.

In [ ]:
df = pd.DataFrame(datos, columns=['ID', 'Título', 'Autor', 'Palabras clave', 'Descargas', 'Resumen'])

Pandas se lleva muy bien con Jupyter Notebooks y con Google Colab.

In [ ]:
df

,ID,Título,Autor,Palabras clave,Descargas,Resumen
0,009004,Comparing two experimental designs for the stu...,Laura Stigliano,"islands, subject islands, subjects, experiment...",74,This paper investigates the phenomenon of subj...
1,008762,Clausal doubling produces phantom islands,Carlos Muñoz Pérez,"phantom islands, clausal doubling, contrastive...",336,Clausal doubling in Spanish seemingly displays...


En este punto, podríamos postular una iteración para que se aplique la función *get_page()* y para que los resultados se agreguen automáticamente a un DataFrame. No la vamos a correr porque (i) no queremos extraer datos de Lingbuzz otra vez (tarda un rato largo), y (ii) va a tirar error en algún momento.

In [ ]:
# Tip: podemos crear una lista de números usando la función range()

for num in range(10,20):
  print(num)

Bien, una vez que (en teoría) tenemos un DataFrame con los datos de todos los manuscritos, procedemos a guardarlo en un archivo CSV (comma separated values).

In [ ]:
df.to_csv('tabla_con_datos.csv', index=False)